# Snowflake Summit 2025 DE107 - Orchestrating Data Analytics Workloads with SQL and Python

### Learning Objective:

- Ease of bringing data into Snowflake
  - Participants will learn how to read files (CSV, Parquet, Excel) using Python and extracting information from those files 
- The power of Python UDFs and common use cases for them
  - Participants will learn about when they should use a UDF, UDTF, UDAF, and Stored Procedure for building their data pipeline
  - Participants will learn about external access and how to make a basic web request in a Python UDF
- The benefits of writing queries in Python and the basics of (pandas) DataFrames
  - Participants will learn about the modularity and composability of DataFrames: a DataFrame can be imported to multiple files, shared across projects, and tested individually. 
  - Participants will learn that iterative and programmatic specification of data transformation is easier in Python than SQL
  - Participants will learn that handling complex datatypes is easier in Python (like working with JSON)
- How to monitor and troubleshoot Python jobs in Snowflake
  - Participants will be introduced to Event Tables and what traces are
  - Participants will learn how to create a basic email notification based on their error logs
- (Optional, if participants finish early): How to schedule a Python stored procedure with Tasks
  - Participants will learn how to schedule their Notebook/SP (#3) with a Task
- (Optional) The extent of the PyData ecosystem and how to incorporate it into their jobs
  - Participants will be introduced to a popular analytics Python package (data viz?) and generating reports using that data


## Adding Python Packages 🎒

Snowflake Notebooks comes pre-installed with common Python libraries for data science 🧪 and machine learning 🧠, such as numpy, pandas, matplotlib, streamlit and more!

If you are looking to use other packages, click on the `Packages` dropdown on the top right to add additional packages to your notebook.

For the purpose of this demo, let's add the following packages: 
- `modin` (version `0.30.1`)
- `openpyxl`
- `s3fs`
- `snowflake-ml-python`

In [ ]:
import streamlit as st
import modin.pandas as pd
import snowflake.snowpark.modin.plugin

## Connecting to Snowflake 

To work with your data in Snowflake, you need to first get a session variable to connect to Snowflake. Since you are already logged in to Snowflake Notebook, you can get your session variable directly through the active notebook session. The session variable is the entrypoint that gives you access to using Snowflake's Python API, including Snowpark.

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Bringing data into Snowflake
You can use pandas on Snowflake to load in [CSV](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/modin/pandas_api/modin.pandas.read_csv#modin.pandas.read_csv), [Parquet](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/modin/pandas_api/modin.pandas.read_parquet#modin.pandas.read_parquet), and [Excel](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/modin/pandas_api/modin.pandas.read_excel#modin.pandas.read_excel) from stage or local file location. Here is the full list of [I/O functionalities supported](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/1.30.0/modin/io).


First let's create a external stage and upload the CSV file. 

In [ ]:
-- Create a external stage (Note: this is not required if running outside of notebook due to EAI)
CREATE OR REPLACE STAGE FROSTBYTES
    URL = 's3://sfquickstarts/frostbyte_tastybytes/';

In [ ]:
menu_item = pd.read_csv("@frostbytes/analytics/menu_item_aggregate_v.csv")

In [ ]:
menu_item.head()

## Profiling and summary statistics
We can look at the size and overall descriptive statistics of our dataframe `menu_item`.  

In [ ]:
menu_item.shape

In [ ]:
menu_item.describe()

In [ ]:
import altair as alt
numeric_cols = ['PRICE', 'BASE_PRICE', 'COST_OF_GOODS_USD', 'COUNT_ORDERS', 'TOTAL_QUANTITY_SOLD']
cols = st.columns(len(numeric_cols))
for idx, col in enumerate(numeric_cols):
    with cols[idx]:
        chart = alt.Chart(menu_item).mark_bar().encode(
            alt.X(f'{col}:Q', bin=True, title=col),
            alt.Y('count():Q', title='Count'),
            tooltip=['count()']
        ).properties(
            width=200,  
            height=300,
            title=f'Distribution of {col}'
        ).configure_title(
            fontSize=14 
        )
        st.altair_chart(chart, use_container_width=True)


## Data Cleaning and Transformation

Now let's clean up the data by performnign some filtering and aggregation.


In [ ]:
# Filter to only records in January 2023
menu_item["DATE"] = pd.to_datetime(menu_item["DATE"])
filtered_menu_item = menu_item[(menu_item["DATE"]>'2023-01-01')&(menu_item["DATE"]<'2023-02-01')]
st.markdown(f'There are {len(menu_item)} rows in the full. After filtering, there are {len(filtered_menu_item)} rows in this daterange.' )

In [ ]:
filtered_menu_item["MENU_ITEM_NAME"].value_counts()

In [ ]:
st.markdown(f'''There are {len(filtered_menu_item["MENU_ITEM_NAME"].unique())} different menu items. \n
That's a lot of different items! Let's see how we can group them into fewer categories.''')

Next, to clean up the data further, we want to to classify menu items into a smaller number of categories. We can do that using Snowflake's Cortex LLM functions [CLASSIFY](https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex). You can use Snowflake Cortex LLM functions via the Snowpark pandas apply function, see examples [here](https://docs.snowflake.com/en/developer-guide/snowpark/python/pandas-on-snowflake#using-snowflake-cortex-llm-functions-with-snowpark-pandas).

In [ ]:
from snowflake.cortex import ClassifyText
filtered_menu_item["MENU_ITEM_CATEGORY"] = filtered_menu_item["MENU_ITEM_NAME"].apply(ClassifyText, categories=["Meal","Dessert","Drinks"])

In [ ]:
filtered_menu_item["MENU_ITEM_CATEGORY"]

Now let's extract the `label` field from the dictionary in the column.

In [ ]:
filtered_menu_item["MENU_ITEM_LABEL"] = filtered_menu_item["MENU_ITEM_CATEGORY"].apply(lambda x: x.get('label'))

In [ ]:
filtered_menu_item["MENU_ITEM_LABEL"]

We can now count the occurrences of each unique value of `MENU_ITEM_LABEL` in the dataframe. We see that most of the records are meals and drinks, with a few desserts and unclassified rows.

In [ ]:
filtered_menu_item["MENU_ITEM_LABEL"].value_counts()

I want to manually examine the records that are unclassified and see what they look like: 

In [ ]:
filtered_menu_item[filtered_menu_item["MENU_ITEM_LABEL"]=="UNCLASSIFIED"]

We are interested in looking specifically at the sales of Buffalo Mac & Cheese across different food trucks. 

In [ ]:
buffalo_mac_cheese = menu_item[menu_item["MENU_ITEM_NAME"]=="Buffalo Mac & Cheese"]

Next, we perform a join to combine our `buffalo_mac_cheese` dataframe with another dataframe `order_item`.

In [ ]:
order_item = pd.read_csv("@frostbytes/analytics/order_item_cost_agg_v.csv")

In [ ]:
order_item

By looking at `order_item`, we see that it has separate month year columns, but `buffalo_mac_cheese` (from the original `menu_item`) has one combined `DATE`, so let's extract the year and month column.

In [ ]:
# Extract year and month to conform with data in `order_item`
buffalo_mac_cheese['YEAR'] = buffalo_mac_cheese['DATE'].dt.year
buffalo_mac_cheese['MONTH'] = buffalo_mac_cheese['DATE'].dt.month

Now let's groupby the year month and menu type.

In [ ]:
# Group by YEAR and MONTH
grouped_bmc = buffalo_mac_cheese.groupby(['YEAR', 'MONTH','MENU_TYPE_ID'])["COUNT_ORDERS","TOTAL_QUANTITY_SOLD"].sum().reset_index()
grouped_bmc

Now that we have the same join key on the two tables, we can merge the columns together.

In [ ]:
# Now merge with order_item on YEAR and MONTH
merged_df = grouped_bmc.merge(order_item, on=['YEAR', 'MONTH'])
merged_df

In [ ]:
st.markdown("This is how the dataframe size changed from performing the merge operation:")
st.markdown(f"order_item size: {order_item.shape} + grouped_bmc size: {grouped_bmc.shape} -> merged_df size: {merged_df.shape}")